zunächst Die Imports der Libraries

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from itertools import islice

Import der Daten
(erstmal nur die ersten 5 Bilder)

In [22]:
currentPath = Path().cwd()

targetPath = currentPath.parent / 'data' / 'KGT_noDefect_simplified'
imsNoF = [[cv2.imread(file, cv2.IMREAD_COLOR), False] for file in islice(targetPath.iterdir(), 5) if file.is_file()]

targetPath = currentPath.parent / 'data' / 'KGT_pitting_simplified'
imsF = [[cv2.imread(file, cv2.IMREAD_COLOR), True] for file in islice(targetPath.iterdir(), 5) if file.is_file()]

ims = imsNoF + imsF

del(imsNoF, imsF, currentPath, targetPath)

zur veranschaulichung Umwandlung in Grey Scale

In [ ]:
imsGray = []
for file, label in ims:
    imsGray.append([cv2.cvtColor(file, cv2.COLOR_BGR2GRAY), label])

finden von Kanten mit Sobel

In [27]:
sobelIms = []
for image, _ in imsGray:
    sobelx = cv2.Sobel(image, cv2.CV_64F, 1, 0, ksize=3)
    sobely = cv2.Sobel(image, cv2.CV_64F, 0, 1, ksize=3)
    sobelIms.append(cv2.convertScaleAbs(cv2.magnitude(sobelx,sobely)))

Für den Moment damit bereits dreifache Trainingsdaten. Kanten halte ich für äußerst hilfreich.

In [28]:
cv2.imshow('Color', ims[0][0])
cv2.imshow('gray', imsGray[0][0])
cv2.imshow('sobel', sobelIms[0])

cv2.waitKey(0)
cv2.destroyAllWindows()